# UFO Question 

Our data science team has predicted that the Earth is going to be invaded by an alien force in the
next years. Our only hope is to replicate a device that can block all alien technology in a radius of
~300km. Sadly, the device was sold in 2004 to an anonymous buyer to protect her hometown and
we don't know how contact her again. We know that the device has been active since 2004 in one
city in the USA, and we want to know where to start our search.
We've included a dataset called ​ufo.csv​. This dataset contains over 80,000 reports of UFO sightings
over the last century (all of them verified by the ESA). Using this dataset, try to guess the city in
which the device has been hidden.


In [1]:
from pymongo import MongoClient

import pandas as pd

import re
import folium

from datetime import datetime

In [2]:
# Select data
data = pd.read_csv('~/Escritorio/Ironhack/lab3/3.4-lab-ufo/ufo.csv')
data.head()

,Unnamed: 0,datetime,city,state,country,shape,duration,total_time,comments,date_posted,latitude,longitude,year,distance
0,0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,2004,1242.667772
1,1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,2005,1325.486319
2,2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,2008,6515.416577
3,3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,2004,1211.971352
4,4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,2004,6960.923396


In [3]:
# conexion con Mongo
cliente=MongoClient()

db=cliente.ufo

alien = db.alien

In [4]:
als = list(alien.find({'$and': [{'country': 'us'}]},
                     {'_id':0,
                     'datetime': 1,
                     'city': 1,
                     'latitude': 1,
                     'longitude': 1,
                     'distance': 1}
                     ))
len(als)

63399

In [5]:
df = pd.DataFrame(als)
df.head()

,datetime,city,latitude,longitude,distance
0,10/10/1949 20:30,san marcos,29.8830556,-97.9411111,1242.66777234
1,10/10/1956 21:00,edna,28.9783333,-96.6458333,1211.9713516
2,10/10/1960 20:00,kaneohe,21.4180556,-157.8036111,6960.92339627
3,10/10/1961 19:00,bristol,36.595,-82.1888889,427.334112543
4,10/10/1965 23:45,norwalk,41.1175,-73.4083333,1295.8277925


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63399 entries, 0 to 63398
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   63399 non-null  object
 1   city       63399 non-null  object
 2   latitude   63399 non-null  object
 3   longitude  63399 non-null  object
 4   distance   63399 non-null  object
dtypes: object(5)
memory usage: 2.4+ MB


In [7]:
df.datetime=df.datetime.str.extract(r'(\d{4})')

In [8]:
df.datetime = df.datetime.astype(int)
df.head()

,datetime,city,latitude,longitude,distance
0,1949,san marcos,29.8830556,-97.9411111,1242.66777234
1,1956,edna,28.9783333,-96.6458333,1211.9713516
2,1960,kaneohe,21.4180556,-157.8036111,6960.92339627
3,1961,bristol,36.595,-82.1888889,427.334112543
4,1965,norwalk,41.1175,-73.4083333,1295.8277925


In [9]:
df=df[df.datetime >= 2004]
df

,datetime,city,latitude,longitude,distance
97,2004,mahwah,41.0886111,-74.1441667,1237.0625821
98,2004,terre haute,39.4666667,-87.4138889,366.596467654
99,2004,indianapolis,39.7683333,-86.1580556,403.237792158
100,2004,morgantown,39.6294444,-79.9561111,721.946493024
101,2004,berkeley,37.8716667,-122.2716667,3131.78205165
...,...,...,...,...,...
63394,2013,woodstock,34.1013889,-84.5194444,318.719194807
63395,2013,boise,43.6136111,-116.2025,2620.02153892
63396,2013,napa,38.2972222,-122.2844444,3126.99141729
63397,2013,vienna,38.9011111,-77.2655556,904.740389278


In [18]:
df['distance'] = df['distance'].astype(float)

In [36]:
df = df[(df.distance > 300) & (df.distance < 305)]
df.reset_index(drop = True, inplace = True)
df

,datetime,city,latitude,longitude,distance
0,2010,oxford,33.6141667,-85.835,302.280898
1,2012,homewood,33.4716667,-86.8008333,301.399697
2,2008,tazewell,36.4541667,-83.5694444,302.824797
3,2007,new tazewell,36.4425,-83.5997222,300.016444
4,2012,olney,38.7308333,-88.0852778,300.431569
5,2012,olney,38.7308333,-88.0852778,300.431569
6,2012,oxford,33.6141667,-85.835,302.280898
7,2013,manchester,37.1536111,-83.7619444,302.476051
8,2011,tazewell,36.4541667,-83.5694444,302.824797
9,2005,hueytown,33.4511111,-86.9966667,303.532720


In [37]:
alien.geoloc.insert_many(df.to_dict('records'))


In [38]:
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [41]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                tiles='openstreetmap', zoom_start=zoom_start)

    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], popup=str(df.city[i]), 
                             radius=1, icon=folium.Icon()).add_to(mapa)

    #mapa.save('images/geomap.html')
    
    
    return mapa

In [45]:
mapa=geomap(df.loc[:1500])
mapa

Parece que la ciudad puede ser Nashville, pues es la ciudad más grande que se encuentra en el medio del círculo.